In [2]:
import tensorflow as tf

from cfg import MAX_CAPTCHA, CHAR_SET_LEN, tb_log_path, save_model
from cnn_sys import crack_captcha_cnn, Y, keep_prob, X
from data_iter import get_next_batch


def train_crack_captcha_cnn():
    """
    训练模型
    :return:
    """
    output = crack_captcha_cnn()
    predict = tf.reshape(output, [-1, MAX_CAPTCHA, CHAR_SET_LEN])  # 36行，4列
    label = tf.reshape(Y, [-1, MAX_CAPTCHA, CHAR_SET_LEN])

    max_idx_p = tf.argmax(predict, 2)  # shape:batch_size,4,nb_cls
    max_idx_l = tf.argmax(label, 2)
    correct_pred = tf.equal(max_idx_p, max_idx_l)

    with tf.name_scope('my_monitor'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=label))
        # loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=Y))
    tf.summary.scalar('my_loss', loss)
    # 最后一层用来分类的softmax和sigmoid有什么不同？

    # optimizer 为了加快训练 learning_rate应该开始大，然后慢慢衰
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    with tf.name_scope('my_monitor'):
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar('my_accuracy', accuracy)

    saver = tf.train.Saver()  # 将训练过程进行保存

    sess = tf.InteractiveSession(
        config=tf.ConfigProto(
            log_device_placement=False
        )
    )

    sess.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tb_log_path, sess.graph)

    step = 0
    while True:
        batch_x, batch_y = get_next_batch(64)  # 64
        _, loss_ = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.95})
        print(step, 'loss:\t', loss_)

        step += 1

        # 每2000步保存一次实验结果
        if step % 700 == 0:
            saver.save(sess, save_model, global_step=step)

        # 在测试集上计算精度
        if step % 50 != 0:
            continue

        # 每50 step计算一次准确率，使用新生成的数据
        batch_x_test, batch_y_test = get_next_batch(256)  # 新生成的数据集个来做测试
        acc = sess.run(accuracy, feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})
        print(step, 'acc---------------------------------\t', acc)

        # 终止条件
        if acc > 0.93:
            break

        # 启用监控 tensor board
        summary = sess.run(merged, feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})
        writer.add_summary(summary, step)


if __name__ == '__main__':
    train_crack_captcha_cnn()
    print('end')
    pass

0 loss:	 3.59414
1 loss:	 3.60334
2 loss:	 3.58491
3 loss:	 3.60842
4 loss:	 3.60226
5 loss:	 3.58361
6 loss:	 3.58683
7 loss:	 3.58523
8 loss:	 3.58884
9 loss:	 3.59983
10 loss:	 3.60001
11 loss:	 3.62318
12 loss:	 3.58348
13 loss:	 3.58951
14 loss:	 3.59461
15 loss:	 3.60557
16 loss:	 3.58801
17 loss:	 3.58165
18 loss:	 3.58471
19 loss:	 3.58451
20 loss:	 3.58601
21 loss:	 3.57983
22 loss:	 3.58854
23 loss:	 3.59493
24 loss:	 3.58823
25 loss:	 3.59748
26 loss:	 3.5901
27 loss:	 3.58077
28 loss:	 3.59526
29 loss:	 3.58634
30 loss:	 3.58782
31 loss:	 3.59345
32 loss:	 3.57898
33 loss:	 3.59674
34 loss:	 3.57772
35 loss:	 3.59037
36 loss:	 3.57134
37 loss:	 3.59328
38 loss:	 3.58163
39 loss:	 3.58186
40 loss:	 3.59226
41 loss:	 3.59122
42 loss:	 3.58518
43 loss:	 3.5899
44 loss:	 3.59774
45 loss:	 3.58501
46 loss:	 3.58771
47 loss:	 3.58895
48 loss:	 3.58974
49 loss:	 3.59411
50 acc---------------------------------	 0.0214844
50 loss:	 3.59052
51 loss:	 3.59337
52 loss:	 3.58374
53 loss

418 loss:	 3.58159
419 loss:	 3.58184
420 loss:	 3.58362
421 loss:	 3.59363
422 loss:	 3.58936
423 loss:	 3.58995
424 loss:	 3.585
425 loss:	 3.58019
426 loss:	 3.58459
427 loss:	 3.58741
428 loss:	 3.58277
429 loss:	 3.58222
430 loss:	 3.5861
431 loss:	 3.5858
432 loss:	 3.5788
433 loss:	 3.58254
434 loss:	 3.57814
435 loss:	 3.57976
436 loss:	 3.58072
437 loss:	 3.58874
438 loss:	 3.58809
439 loss:	 3.58641
440 loss:	 3.58732
441 loss:	 3.58884
442 loss:	 3.58071
443 loss:	 3.58085
444 loss:	 3.58394
445 loss:	 3.58241
446 loss:	 3.58732
447 loss:	 3.58536
448 loss:	 3.58693
449 loss:	 3.58327
450 acc---------------------------------	 0.0322266
450 loss:	 3.58603
451 loss:	 3.58675
452 loss:	 3.58324
453 loss:	 3.58135
454 loss:	 3.5863
455 loss:	 3.58283
456 loss:	 3.58268
457 loss:	 3.58699
458 loss:	 3.5838
459 loss:	 3.59003
460 loss:	 3.58712
461 loss:	 3.58233
462 loss:	 3.58091
463 loss:	 3.58578
464 loss:	 3.58291
465 loss:	 3.59143
466 loss:	 3.58621
467 loss:	 3.59062
468 l

831 loss:	 3.58503
832 loss:	 3.5871
833 loss:	 3.58315
834 loss:	 3.58256
835 loss:	 3.58423
836 loss:	 3.58197
837 loss:	 3.58489
838 loss:	 3.58171
839 loss:	 3.57888
840 loss:	 3.58323
841 loss:	 3.58413
842 loss:	 3.58218
843 loss:	 3.58393
844 loss:	 3.58567
845 loss:	 3.58194
846 loss:	 3.58283
847 loss:	 3.58291
848 loss:	 3.58227
849 loss:	 3.58362
850 acc---------------------------------	 0.0332031
850 loss:	 3.58631
851 loss:	 3.58256
852 loss:	 3.58018
853 loss:	 3.5842
854 loss:	 3.58148
855 loss:	 3.58574
856 loss:	 3.58349
857 loss:	 3.58452
858 loss:	 3.58676
859 loss:	 3.5841
860 loss:	 3.58095
861 loss:	 3.58367
862 loss:	 3.58601
863 loss:	 3.58801
864 loss:	 3.58525
865 loss:	 3.58563
866 loss:	 3.58529
867 loss:	 3.58361
868 loss:	 3.58411
869 loss:	 3.58421
870 loss:	 3.58031
871 loss:	 3.5784
872 loss:	 3.58335
873 loss:	 3.58629
874 loss:	 3.58303
875 loss:	 3.58468
876 loss:	 3.57922
877 loss:	 3.58162
878 loss:	 3.5844
879 loss:	 3.58278
880 loss:	 3.58533
881

1231 loss:	 3.58479
1232 loss:	 3.58831
1233 loss:	 3.58522
1234 loss:	 3.58393
1235 loss:	 3.58558
1236 loss:	 3.58389
1237 loss:	 3.58378
1238 loss:	 3.58446
1239 loss:	 3.58559
1240 loss:	 3.58515
1241 loss:	 3.58507
1242 loss:	 3.58406
1243 loss:	 3.58523
1244 loss:	 3.58523
1245 loss:	 3.58436
1246 loss:	 3.58506
1247 loss:	 3.58428
1248 loss:	 3.58581
1249 loss:	 3.58886
1250 acc---------------------------------	 0.0234375
1250 loss:	 3.58175
1251 loss:	 3.58342
1252 loss:	 3.58314
1253 loss:	 3.58383
1254 loss:	 3.58317
1255 loss:	 3.58435
1256 loss:	 3.58206
1257 loss:	 3.57959
1258 loss:	 3.58732
1259 loss:	 3.58067
1260 loss:	 3.58141
1261 loss:	 3.58441
1262 loss:	 3.57878
1263 loss:	 3.58369
1264 loss:	 3.58183
1265 loss:	 3.58301
1266 loss:	 3.58595
1267 loss:	 3.58148
1268 loss:	 3.58701
1269 loss:	 3.58439
1270 loss:	 3.58401
1271 loss:	 3.58419
1272 loss:	 3.58181
1273 loss:	 3.58218
1274 loss:	 3.58239
1275 loss:	 3.58653
1276 loss:	 3.58443
1277 loss:	 3.58522
1278 lo

1622 loss:	 3.58499
1623 loss:	 3.58193
1624 loss:	 3.58258
1625 loss:	 3.58267
1626 loss:	 3.58277
1627 loss:	 3.58515
1628 loss:	 3.58159
1629 loss:	 3.58542
1630 loss:	 3.58483
1631 loss:	 3.58699
1632 loss:	 3.58716
1633 loss:	 3.58508
1634 loss:	 3.58637
1635 loss:	 3.58125
1636 loss:	 3.57962
1637 loss:	 3.58461
1638 loss:	 3.58325
1639 loss:	 3.58832
1640 loss:	 3.58237
1641 loss:	 3.58407
1642 loss:	 3.57973
1643 loss:	 3.58362
1644 loss:	 3.58697
1645 loss:	 3.5877
1646 loss:	 3.58681
1647 loss:	 3.58838
1648 loss:	 3.58276
1649 loss:	 3.58324
1650 acc---------------------------------	 0.0273438
1650 loss:	 3.58087
1651 loss:	 3.58553
1652 loss:	 3.58296
1653 loss:	 3.58297
1654 loss:	 3.58557
1655 loss:	 3.5866
1656 loss:	 3.58526
1657 loss:	 3.58539
1658 loss:	 3.58274
1659 loss:	 3.58686
1660 loss:	 3.58515
1661 loss:	 3.58284
1662 loss:	 3.58175
1663 loss:	 3.5874
1664 loss:	 3.5813
1665 loss:	 3.58271
1666 loss:	 3.58407
1667 loss:	 3.57976
1668 loss:	 3.58041
1669 loss:	

...

5149 loss:	 1.65664
5150 acc---------------------------------	 0.555664
5150 loss:	 1.64298
5151 loss:	 1.57255
5152 loss:	 1.65946
5153 loss:	 1.73012
5154 loss:	 1.70969
5155 loss:	 1.56938
5156 loss:	 1.76373
5157 loss:	 1.59726
5158 loss:	 1.69604
5159 loss:	 2.06579
5160 loss:	 1.65384
5161 loss:	 1.74546
5162 loss:	 1.72698
5163 loss:	 1.73598
5164 loss:	 1.64774
5165 loss:	 1.66505
5166 loss:	 1.71695
5167 loss:	 1.73601
5168 loss:	 1.56984
5169 loss:	 1.64755
5170 loss:	 1.60538
5171 loss:	 1.71052
5172 loss:	 1.68495
5173 loss:	 1.76498
5174 loss:	 1.7898
5175 loss:	 1.7405
5176 loss:	 1.71357
5177 loss:	 1.74541
5178 loss:	 1.56538
5179 loss:	 1.71594
5180 loss:	 1.70848
5181 loss:	 1.57886
5182 loss:	 1.85681
5183 loss:	 1.60294
5184 loss:	 1.70287
5185 loss:	 1.66046
5186 loss:	 1.5866
5187 loss:	 1.84689
5188 loss:	 1.60889
5189 loss:	 1.61194
5190 loss:	 1.64332
5191 loss:	 1.59924
5192 loss:	 1.64178
5193 loss:	 1.67911
5194 loss:	 1.68884
5195 loss:	 1.80602
5196 loss:	

5540 loss:	 1.64632
5541 loss:	 1.58017
5542 loss:	 1.48603
5543 loss:	 1.58088
5544 loss:	 1.53054
5545 loss:	 1.70829
5546 loss:	 1.75096
5547 loss:	 1.61361
5548 loss:	 1.64198
5549 loss:	 1.68716
5550 acc---------------------------------	 0.566406
5550 loss:	 1.60285
5551 loss:	 1.54375
5552 loss:	 1.66374
5553 loss:	 1.63538
5554 loss:	 1.64889
5555 loss:	 1.58274
5556 loss:	 1.6711
5557 loss:	 1.62179
5558 loss:	 1.504
5559 loss:	 1.58878
5560 loss:	 1.39814
5561 loss:	 1.53125
5562 loss:	 1.61966
5563 loss:	 1.63662
5564 loss:	 1.6881
5565 loss:	 1.64877
5566 loss:	 1.71246
5567 loss:	 1.68097
5568 loss:	 1.64157
5569 loss:	 1.59004
5570 loss:	 1.59991
5571 loss:	 1.59675
5572 loss:	 1.65271
5573 loss:	 1.7709
5574 loss:	 1.74023
5575 loss:	 1.72373
5576 loss:	 1.58176
5577 loss:	 1.67027
5578 loss:	 1.62553
5579 loss:	 1.53873
5580 loss:	 1.64522
5581 loss:	 1.75992
5582 loss:	 1.52977
5583 loss:	 1.54847
5584 loss:	 1.56501
5585 loss:	 1.66022
5586 loss:	 1.71143
5587 loss:	 1

5932 loss:	 1.52114
5933 loss:	 1.67882
5934 loss:	 1.69539
5935 loss:	 1.57914
5936 loss:	 1.59523
5937 loss:	 1.62543
5938 loss:	 1.54975
5939 loss:	 1.66676
5940 loss:	 1.55658
5941 loss:	 1.52889
5942 loss:	 1.50858
5943 loss:	 1.55822
5944 loss:	 1.50734
5945 loss:	 1.774
5946 loss:	 1.50332
5947 loss:	 1.59374
5948 loss:	 1.47907
5949 loss:	 1.52454
5950 acc---------------------------------	 0.611328
5950 loss:	 1.58252
5951 loss:	 1.74217
5952 loss:	 1.60126
5953 loss:	 1.52197
5954 loss:	 1.56115
5955 loss:	 1.49989
5956 loss:	 1.55238
5957 loss:	 1.5518
5958 loss:	 1.63487
5959 loss:	 1.70873
5960 loss:	 1.54601
5961 loss:	 1.55994
5962 loss:	 1.64067
5963 loss:	 1.44559
5964 loss:	 1.52019
5965 loss:	 1.64643
5966 loss:	 1.62182
5967 loss:	 1.52957
5968 loss:	 1.6015
5969 loss:	 1.50189
5970 loss:	 1.57822
5971 loss:	 1.57154
5972 loss:	 1.561
5973 loss:	 1.56701
5974 loss:	 1.51471
5975 loss:	 1.54028
5976 loss:	 1.37902
5977 loss:	 1.52168
5978 loss:	 1.63706
5979 loss:	 1.

6324 loss:	 1.49785
6325 loss:	 1.40102
6326 loss:	 1.42663
6327 loss:	 1.53879
6328 loss:	 1.38006
6329 loss:	 1.53821
6330 loss:	 1.31318
6331 loss:	 1.46089
6332 loss:	 1.37537
6333 loss:	 1.47507
6334 loss:	 1.51697
6335 loss:	 1.47497
6336 loss:	 1.66391
6337 loss:	 1.55182
6338 loss:	 1.39794
6339 loss:	 1.57013
6340 loss:	 1.5529
6341 loss:	 1.52151
6342 loss:	 1.39677
6343 loss:	 1.51584
6344 loss:	 1.58695
6345 loss:	 1.58095
6346 loss:	 1.56147
6347 loss:	 1.60375
6348 loss:	 1.60516
6349 loss:	 1.45556
6350 acc---------------------------------	 0.587891
6350 loss:	 1.65874
6351 loss:	 1.49127
6352 loss:	 1.59635
6353 loss:	 1.42897
6354 loss:	 1.58824
6355 loss:	 1.4776
6356 loss:	 1.60277
6357 loss:	 1.51915
6358 loss:	 1.57106
6359 loss:	 1.53231
6360 loss:	 1.49801
6361 loss:	 1.61688
6362 loss:	 1.50207
6363 loss:	 1.57018
6364 loss:	 1.40383
6365 loss:	 1.44262
6366 loss:	 1.59459
6367 loss:	 1.49163
6368 loss:	 1.4386
6369 loss:	 1.76447
6370 loss:	 1.6302
6371 loss:	 

KeyboardInterrupt: 